# Preprocessing NLP per ticket di supporto

In questo notebook vengono preparati i dati testuali dei ticket per l'addestramento dei modelli di classificazione:
- costruzione del campo testuale (oggetto + descrizione)
- pulizia testuale semplice e riproducibile
- analisi di base delle lunghezze
- suddivisione train/test senza leakage

In [1]:
import pandas as pd
import re

from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"

from sklearn.model_selection import train_test_split

pd.set_option("display.max_colwidth", 200)

df = pd.read_csv(DATA_DIR / "raw" / "tickets_realistic.csv")

print("Shape iniziale:", df.shape)
df.head()

Shape iniziale: (4800, 7)


,id,title,body,category,priority,title_length,body_length
0,1,Blocco sistema durante elaborazione fattura 7397/23 per Pisacane-Zaccagnini s.r.l.,Monitoraggio passivo dei log modulo gestione utenti per statistiche utilizzo. Analisi a posteriori dei tracciati per identificare trend di lungo periodo. Non ci sono errori attivi o malfunzionamen...,Amministrazione,media,82,289
1,2,Batch: consolidamento dati contabili,Nota di update anagrafica per cliente Pavone s.r.l. nel system clienti. Inseriti dati nuovo referente giulia.verdi a puro titolo informativo per il database. Nessuna azione di sincronizzazione ric...,Amministrazione,bassa,36,313
2,3,Discrepanza contabile: bonifico 452.61 non visibile (chiusura bloccata),Malfunzionamento export Excel per report mensile. La generazione del file si interrompe a metà con err generico. I dati a video sono corretti. Serve il file esportato per riunione di domani mattin...,Amministrazione,alta,71,239
3,4,Rallentamento fatturazione: salvataggio impiega 40 secondi (troppo),Nota di aggiornamento anagrafica per cliente Alighieri e figli nel sys. reporitstica. Inseriti dati nuovo referente mario.rossi a puro titolo informativo per il database. Nessuna azione di sincron...,Amministrazione,media,67,288
4,5,Backup: storico fatture 2004 - negoziazione,Nota di aggiornamento anagrafica per cliente Cherubini-Mazzacurati Group nel sistema vendite. Inseriti dati nuovo refreente admin_01 a puro titolo informativo per il database. Nsesuna azione di si...,Amministrazione,bassa,43,293


## 2. Costruzione del campo testuale unico

Vengono uniti oggetto e descrizione del ticket in un singolo campo `text`, che sarà utilizzato come input testuale per tutti i modelli.


In [2]:
TITLE_COL = "title"      
BODY_COL  = "body"       

# Controllo che le colonne esistano
print("Colonne presenti:", df.columns.tolist())
if TITLE_COL not in df.columns or BODY_COL not in df.columns:
    raise ValueError(f"Le colonne {TITLE_COL} e/o {BODY_COL} non esistono nel dataset. Adattale prima di proseguire.")

# Costruzione del campo testuale
df["text"] = (
    df[TITLE_COL].fillna("").astype(str) + " " + df[BODY_COL].fillna("").astype(str)
).str.strip()

# Rimuovo righe con testo completamente vuoto
before = df.shape[0]
df = df[df["text"].str.len() > 0].copy()
after = df.shape[0]

print(f"Righe totali dopo rimozione testi vuoti: {after} (rimosse {before - after})")
df[["id", TITLE_COL, BODY_COL, "text"]].head()


Colonne presenti: ['id', 'title', 'body', 'category', 'priority', 'title_length', 'body_length']
Righe totali dopo rimozione testi vuoti: 4800 (rimosse 0)


,id,title,body,text
0,1,Blocco sistema durante elaborazione fattura 7397/23 per Pisacane-Zaccagnini s.r.l.,Monitoraggio passivo dei log modulo gestione utenti per statistiche utilizzo. Analisi a posteriori dei tracciati per identificare trend di lungo periodo. Non ci sono errori attivi o malfunzionamen...,Blocco sistema durante elaborazione fattura 7397/23 per Pisacane-Zaccagnini s.r.l. Monitoraggio passivo dei log modulo gestione utenti per statistiche utilizzo. Analisi a posteriori dei tracciati ...
1,2,Batch: consolidamento dati contabili,Nota di update anagrafica per cliente Pavone s.r.l. nel system clienti. Inseriti dati nuovo referente giulia.verdi a puro titolo informativo per il database. Nessuna azione di sincronizzazione ric...,Batch: consolidamento dati contabili Nota di update anagrafica per cliente Pavone s.r.l. nel system clienti. Inseriti dati nuovo referente giulia.verdi a puro titolo informativo per il database. N...
2,3,Discrepanza contabile: bonifico 452.61 non visibile (chiusura bloccata),Malfunzionamento export Excel per report mensile. La generazione del file si interrompe a metà con err generico. I dati a video sono corretti. Serve il file esportato per riunione di domani mattin...,Discrepanza contabile: bonifico 452.61 non visibile (chiusura bloccata) Malfunzionamento export Excel per report mensile. La generazione del file si interrompe a metà con err generico. I dati a vi...
3,4,Rallentamento fatturazione: salvataggio impiega 40 secondi (troppo),Nota di aggiornamento anagrafica per cliente Alighieri e figli nel sys. reporitstica. Inseriti dati nuovo referente mario.rossi a puro titolo informativo per il database. Nessuna azione di sincron...,Rallentamento fatturazione: salvataggio impiega 40 secondi (troppo) Nota di aggiornamento anagrafica per cliente Alighieri e figli nel sys. reporitstica. Inseriti dati nuovo referente mario.rossi ...
4,5,Backup: storico fatture 2004 - negoziazione,Nota di aggiornamento anagrafica per cliente Cherubini-Mazzacurati Group nel sistema vendite. Inseriti dati nuovo refreente admin_01 a puro titolo informativo per il database. Nsesuna azione di si...,Backup: storico fatture 2004 - negoziazione Nota di aggiornamento anagrafica per cliente Cherubini-Mazzacurati Group nel sistema vendite. Inseriti dati nuovo refreente admin_01 a puro titolo infor...


## 3. Pulizia testuale semplice

Viene definita una funzione `clean_text` che applica:

- conversione in minuscolo
- rimozione dei simboli non utili
- normalizzazione degli spazi


In [3]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        text = "" if text is None else str(text)
        
    # minuscole (e togli lowercase=False dal TFIDF)
    text = text.lower()
    
    # rimuovi solo simboli non utili
    text = re.sub(r"[^\w\sàèéìòù]", " ", text)
    
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Test veloce della funzione su un esempio
example = "URGENTE: Fattura 12345 non pagata!!!"
print("Originale:", example)
print("Pulito   :", clean_text(example))


Originale: URGENTE: Fattura 12345 non pagata!!!
Pulito   : urgente fattura 12345 non pagata


## 4. Applicazione della pulizia e controlli di base

Applicazione di `clean_text` al campo `text` per ottenere `text_clean` e verifica dei primi 5 record prima/dopo


In [4]:
df["text_clean"] = df["text"].apply(clean_text)

print("Esempi di testo prima/dopo:\n")
for i in range(5):
    print(f"--- Ticket {i} ---")
    print("TEXT      :", df.iloc[i]["text"])
    print("TEXT_CLEAN:", df.iloc[i]["text_clean"])
    print()


Esempi di testo prima/dopo:

--- Ticket 0 ---
TEXT      : Blocco sistema durante elaborazione fattura 7397/23 per Pisacane-Zaccagnini s.r.l. Monitoraggio passivo dei log modulo gestione utenti per statistiche utilizzo. Analisi a posteriori dei tracciati per identificare trend di lungo periodo. Non ci sono errori attivi o malfunzionamenti. Attività di studio per future ottimizzazioni, senza impatto attuale., problema su utenteo
TEXT_CLEAN: blocco sistema durante elaborazione fattura 7397 23 per pisacane zaccagnini s r l monitoraggio passivo dei log modulo gestione utenti per statistiche utilizzo analisi a posteriori dei tracciati per identificare trend di lungo periodo non ci sono errori attivi o malfunzionamenti attività di studio per future ottimizzazioni senza impatto attuale problema su utenteo

--- Ticket 1 ---
TEXT      : Batch: consolidamento dati contabili Nota di update anagrafica per cliente Pavone s.r.l. nel system clienti. Inseriti dati nuovo referente giulia.verdi a puro ti

## 5. Analisi delle lunghezze dei testi

Analisi la lunghezza dei testi (in parole) prima e dopo la pulizia al fine di capire quanto contesto avranno a disposizione i modelli


In [5]:
# numero di parole prima e dopo la pulizia
df["len_words_raw"] = df["text"].str.split().str.len()
df["len_words_clean"] = df["text_clean"].str.split().str.len()

summary = df[["len_words_raw", "len_words_clean"]].describe().T
print(summary)

# controllo ticket estremamente corti
print("\nEsempi di ticket molto corti (<= 3 parole pulite):\n")
short_mask = df["len_words_clean"] <= 3
df[short_mask][["id", "text", "text_clean", "len_words_clean"]].head(10)


                  count       mean       std   min   25%   50%   75%   max
len_words_raw    4800.0  46.846458  7.337769  30.0  41.0  46.0  52.0  81.0
len_words_clean  4800.0  47.941042  7.779043  30.0  42.0  47.0  53.0  86.0

Esempi di ticket molto corti (<= 3 parole pulite):



,id,text,text_clean,len_words_clean


## 6. Suddivisione train/test

Utilizzo di `train_test_split` scikit-learn per dividere il dataset in due parti (training set e test set) sul testo grezzo (`text`) e sulle etichette:

- `category` (classificazione della tipologia di ticket)
- `priority` (bassa / media / alta)

Uso di `stratify` sulla categoria per mantenere una distribuzione simile nel train e nel test.


In [6]:
 # Controllo che le colonne target esistano
for col in ["category", "priority"]:
    if col not in df.columns:
        raise ValueError(f"La colonna target '{col}' non esiste nel dataset. Controlla il generatore di dataset.")

X = df["text"] 

y_cat = df["category"]
y_pri = df["priority"]

X_train, X_test, y_cat_train, y_cat_test = train_test_split(
    X,
    y_cat,
    test_size=0.2,
    random_state=42,
    stratify=y_cat
)

# riallineo anche le priorità usando gli indici
y_pri_train = df.loc[X_train.index, "priority"]
y_pri_test  = df.loc[X_test.index,  "priority"]

print("Dimensioni train/test:")
print("X_train:", X_train.shape[0])
print("X_test :", X_test.shape[0])

Dimensioni train/test:
X_train: 3840
X_test : 960


## 7. Controllo della distribuzione delle classi

Verifica della distribuzione categorie affinchè sia simile nel dataset completo, nel train e nel test.


In [7]:
def print_distribution_and_counts(label, series_full, series_train, series_test):
    """Stampa distribuzioni percentuali e conteggi assoluti per full/train/test di una variabile target."""
    
    datasets = {
        "full" : series_full,
        "train": series_train,
        "test" : series_test
    }
    
    print(f"\n=== {label.upper()} ===")

    # Distribuzioni percentuali
    print("\nDistribuzioni percentuali (%):")
    for name, s in datasets.items():
        perc = s.value_counts(normalize=True).mul(100).round(2)
        print(f"\n{name}:")
        print(perc)

    # Conteggi assoluti
    print("\nConteggi assoluti:")
    for name, s in datasets.items():
        print(f"\n{name}:")
        print(s.value_counts())


# Uso della funzione
print_distribution_and_counts("Category", y_cat, y_cat_train, y_cat_test)
print_distribution_and_counts("Priority", y_pri, y_pri_train, y_pri_test)



=== CATEGORY ===

Distribuzioni percentuali (%):

full:
category
Tecnico            45.0
Amministrazione    30.0
Commerciale        25.0
Name: proportion, dtype: float64

train:
category
Tecnico            45.0
Amministrazione    30.0
Commerciale        25.0
Name: proportion, dtype: float64

test:
category
Tecnico            45.0
Amministrazione    30.0
Commerciale        25.0
Name: proportion, dtype: float64

Conteggi assoluti:

full:
category
Tecnico            2160
Amministrazione    1440
Commerciale        1200
Name: count, dtype: int64

train:
category
Tecnico            1728
Amministrazione    1152
Commerciale         960
Name: count, dtype: int64

test:
category
Tecnico            432
Amministrazione    288
Commerciale        240
Name: count, dtype: int64

=== PRIORITY ===

Distribuzioni percentuali (%):

full:
priority
media    37.25
alta     33.75
bassa    29.00
Name: proportion, dtype: float64

train:
priority
media    37.50
alta     34.66
bassa    27.84
Name: proportion, dt

## 8. Salvataggio del dataset con indicazione train/test

Viene aggiunta una colonna `split` al DataFrame (`train` / `test`) e salvato un CSV aggiornato, che sarà il punto di partenza per EDA e training.


In [8]:
# inizialmente tutto train
df["split"] = "train"

# assegno 'test' alle righe presenti in X_test
df.loc[X_test.index, "split"] = "test"

print(df["split"].value_counts())

# TODO: adatta il path/output
output_path = DATA_DIR / "splits" / "tickets_preprocessed_split.csv"
df.to_csv(output_path, index=False)
print(f"\nFile salvato in: {output_path}")


split
train    3840
test      960
Name: count, dtype: int64

File salvato in: C:\project-work\data\splits\tickets_preprocessed_split.csv
